In [11]:
import os
os.chdir('..')

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    artifact_dir:Path
    tensorboard_log_dir:Path
    checkpoint_model_filepath:Path

In [13]:
from pathlib import Path
from DeepClassifier.utils import read_yaml_file, make_directories
from DeepClassifier.constants import *
# from DeepClassifier.entity import PrepareCallbacksConfig

class ConfigurationManager:
    def __init__(
            self,
            config_filepath:Path=CONFIG_FILEPATH,
            params_filepath:Path=PARAMS_FILEPATH
        ) -> None:
        self.config = read_yaml_file(config_filepath)
        make_directories([self.config.root_artifact_dir])
        self.params = read_yaml_file(params_filepath)

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        prepare_callbacks_config_info = self.config.prepare_callbacks

        make_directories([
                Path(prepare_callbacks_config_info.artifact_dir),
                Path(prepare_callbacks_config_info.tensorboard_log_dir),
                Path(os.path.dirname(prepare_callbacks_config_info.checkpoint_model_filepath))
            ])

        prepare_callbacks_config = PrepareCallbacksConfig(
                                        artifact_dir=Path(prepare_callbacks_config_info.artifact_dir),
                                        tensorboard_log_dir=Path(prepare_callbacks_config_info.tensorboard_log_dir),
                                        checkpoint_model_filepath=Path(prepare_callbacks_config_info.checkpoint_model_filepath)
                                    )

        return prepare_callbacks_config


In [14]:
# from DeepClassifier.entity import PrepareCallbacksConfig
import tensorflow as tf
import time

class PrepareCallbacks:
    def __init__(self, config:PrepareCallbacksConfig) -> None:
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        log_dir = os.path.join(
            self.config.tensorboard_log_dir, 
            f"tb_logs_at_timestamp"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=log_dir)


    @property
    def _create_ckpt_callbacks(self):
        # timestamp = time.strftime("")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
            mode='auto'
        )

    @property
    def _create_early_stopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='auto',
            patience=10,
            restore_best_weights=True
        )

    def get_callbacks(self) -> list:
        callbacks = [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

        return callbacks

In [15]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(prepare_callbacks_config)
    prepare_callbacks.get_callbacks()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'configs\\config.yaml'